In [ ]:
from ipyleaflet import Map, basemaps, Marker, WidgetControl, projections, ImageOverlay, GeoData, LayersControl, DrawControl

from ipywidgets import Layout, HBox, VBox
import ipywidgets as widgets

from ipyfilechooser import FileChooser

from shapely import Point

from matplotlib import cm, colors

from PIL import Image
from pathlib import Path
import geopandas as gp
import numpy as np
import xarray as xr
import rioxarray as rxr
import rasterio
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import pandas as pd

from glacier_flow_tools.pathlines import compute_pathline, pathline_to_geopandas_dataframe, series_to_pathline_geopandas_dataframe
from glacier_flow_tools.utils import qgis2cmap, tqdm_joblib
from glacier_flow_tools.geom import distances

In [ ]:
center = [72, -45]
zoom = 2.75
crs = "EPSG:3413"
qgis_colormap = Path("../data/speed-colorblind.txt")
cmap = qgis2cmap(qgis_colormap, name="speeds")
vmin, vmax = [10, 1500]

## Select a raster file that can be opened with xarray

In [ ]:
# Create and display a FileChooser widget
raster_file_upload = FileChooser('.')
display(raster_file_upload)

In [ ]:
raster_url = raster_file_upload.selected
ds = xr.open_dataset(raster_url)
ds

In [ ]:
ds.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=True)
ds.rio.write_crs(crs, inplace=True)

In [ ]:
minx, maxx, miny, maxy = np.min(ds["x"]), np.max(ds["x"]), np.min(ds["y"]), np.max(ds["y"])
assert np.all(np.diff(ds["x"])) and np.all(np.diff(ds["y"])) 
resolution = (ds.x[1]-ds.x[0])
src_transform = ds.rio.transform()
dst_transform = rasterio.transform.from_origin(minx, maxy,
                                               resolution, resolution)

In [ ]:
            # allocate for output warped image
            dst_width = int((maxx - minx)//resolution)
            dst_height = int((maxy - miny)//resolution)
            dst_data = np.zeros((dst_height, dst_width), dtype=np.float32)
            print(dst_data)
            # warp image to output resolution
            rasterio.warp.reproject(source=ds.values, destination=dst_data,
                src_transform=src_transform,
                src_crs=ds.rio.crs,
                src_nodata=np.nan,
                dst_transform=dst_transform,
                dst_crs=crs,
                dst_resolution=(resolution, resolution))
            # calculate centered coordinates
            transform = dst_transform * dst_transform.translation(0.5, 0.5)
            x_coords, _ = transform * (np.arange(dst_width), np.zeros(dst_width))
            _, y_coords = transform * (np.zeros(dst_height), np.arange(dst_height))

In [ ]:
layout = Layout(width="25%")

vx_select = widgets.Select(
    options=list(ds.data_vars),
    value="vx" if "vx" in ds.data_vars else None,
    # rows=10,
    description='velocity x-component:',
    disabled=False,
    layout=layout
)
vy_select = widgets.Select(
    options=list(ds.data_vars),
    value="vy" if "vy" in ds.data_vars else None,
    # rows=10,
    description='velocity y-component:',
    disabled=False,
    layout=layout
)

display(HBox([vx_select, vy_select]))

In [ ]:
vx_varname = vx_select.value
vy_varname = vy_select.value

Vx = ds[vx_varname]
Vy = ds[vy_varname]
V = (Vx**2 + Vy**2)**(1./2)


vx = np.squeeze(Vx.to_numpy())
vy = np.squeeze(Vy.to_numpy())
x = ds["x"].to_numpy()
y = ds["y"].to_numpy()

x_min, x_max = np.min(x), np.max(x)
y_min, y_max = np.min(y), np.max(y)

norm = colors.Normalize(vmin=vmin, vmax=vmax)
mapper = cm.ScalarMappable(norm=norm, cmap=cmap)

v_rgba = mapper.to_rgba(V.to_numpy())

In [ ]:
map = Image.fromarray((v_rgba * 255).astype(np.uint8), mode="RGBA")
image_file = "speed.png"
map.save(image_file)

In [ ]:
draw_control = DrawControl()
draw_control.marker =  {
    "shapeOptions": {
        "color": "#6bc2e5",
        "weight": 8,
        "opacity": 1.0
    }
}
# Create a list to store the coordinates of the markers
markers = []

# Define a callback function that will be called when a new shape is drawn
def handle_draw(target, action, geo_json):
    if action == 'created' and geo_json['geometry']['type'] == 'Point':
        markers.append(geo_json['geometry']['coordinates'])

# Add the callback function to the DrawControl
draw_control.on_draw(handle_draw)

In [ ]:
image_file = "speed.png"
image = ImageOverlay(url=image_file, name="Speed", opacity=0.5, bounds=[y_min, x_min, y_max, x_max])

m = Map(basemap=basemaps.NASAGIBS.BlueMarble3413, crs=projections.EPSG3413.NASAGIBS, zoom=zoom, center=(72, -45))
m.add(draw_control)


In [ ]:
np.array([k for k, _ in enumerate(markers)])

In [ ]:
starting_points = gp.GeoDataFrame(geometry=[Point(p) for p in markers], crs="EPSG:4326").to_crs(crs)
print(starting_points)

In [ ]:
output = widgets.Output()

dt_select = widgets.FloatText(
    value=1,
    description='Time step:',
    disabled=False
)
total_time_select = widgets.FloatText(
    value=1_000,
    description='Total time:',
    disabled=False
)
reverse_select = widgets.Checkbox(
    value=True,
    description='Reverse direction',
    disabled=False
)
submit_select = widgets.Button(
    description='Start calculation',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Start calculation',
    icon='check'
)

def on_button_clicked(b):
    with output:
        dt = dt_select.value
        total_time = total_time_select.value
        reverse = reverse_select.value
        result = compute_pathline(
                [*starting_point.geometry[0].coords[0]],
                vx,
                vy,
                x,
                y,
                dt=dt,
                total_time=total_time,
                reverse=reverse,
                progress=True,
                progress_kwargs= {"leave": False, "position": 1},
        )

        return result
    
submit_select.on_click(on_button_clicked)
    
display(VBox([dt_select, total_time_select, reverse_select, submit_select]))

In [ ]:
    n_pts = len(starting_points)
    n_jobs = 4
    dt = dt_select.value
    total_time = total_time_select.value
    reverse = reverse_select.value

    with tqdm_joblib(
        tqdm(desc="Processing Pathlines", total=n_pts, leave=True, position=0)
    ) as progress_bar:  # pylint: disable=unused-variable
        futures = Parallel(n_jobs=n_jobs)(
            delayed(compute_pathline)(
                [*df.geometry.coords[0]],
                Vx,
                Vy,
                x,
                y,
                dt=dt,
                total_time=total_time,
                reverse=reverse,
                notebook=True,
                progress=True,
                progress_kwargs={"leave": True, "position": 1},
            )
            for index, df in starting_points.iterrows()
        )
    pathlines_df = pd.concat(
        list(
            starting_points.reset_index().apply(
                series_to_pathline_geopandas_dataframe, pathline=next(iter(futures)), axis=1
            ).reset_index(drop=True)
        )
    ).reset_index(drop=True)


In [ ]:
import shapely
pathlines_epsg4326_df = pathlines_df.to_crs("EPSG:4326")

In [ ]:
lonc, latc = np.mean(shapely.get_coordinates(pathlines_epsg4326_df.geometry), axis=0)

In [ ]:
pathlines_layer = GeoData(
    geo_dataframe=pathlines_epsg4326_df,
    style={
        "color": "r",
        "fillColor": "#3366cc",
        "opacity": 1,
        "weight": 1,
        "dashArray": "2",
        "fillOpacity": 0.6,
    },
    hover_style={"fillColor": "red", "fillOpacity": 0.2},
    name="Pathline",
)

# distance_dict =  dict(zip(pathlines_epsg4326["level_0"].tolist(), pathlines_epsg4326['distance'].tolist()))
#geo_data = json.loads(pathlines_epsg4326.to_json())

from ipyleaflet import Choropleth, MarkerCluster
# layer = Choropleth(
#     geo_data=geo_data,
#     choro_data=distance_dict,
#     key="pathline_id",
#     border_color='black',
#     style={'fillOpacity': 0.8, 'dashArray': '5, 5'})

pathlines_circles = CircleMarker(location=[[p[1], p[0]] for p in shapely.get_coordinates(pathlines_epsg4326_df.geometry)], radius=1000, color="#000000")

pathlines_cluster = MarkerCluster(
    markers=[Marker(location=geolocation.coords[0][::-1]) for geolocation in pathlines_epsg4326_df.geometry])

m = Map(basemap=basemaps.NASAGIBS.BlueMarble3413, crs=projections.EPSG3413.NASAGIBS, zoom=zoom, center=(latc, lonc))
#m.add_layer(pathlines_cluster)
m.add(pathlines_circles)
m.add(LayersControl())

In [ ]:
[[p[1], p[0]] for p in shapely.get_coordinates(pathlines_epsg4326_df.geometry)]

In [ ]:
from ipyleaflet import GeoJSON, Circle, CircleMarker
import json
import requests

In [ ]:
def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)

geo_json_data = load_data(
    'https://raw.githubusercontent.com/jupyter-widgets/ipyleaflet/master/examples/us-states.json',
    'us-states.json',
     json.load)

unemployment = load_data(
    'https://raw.githubusercontent.com/jupyter-widgets/ipyleaflet/master/examples/US_Unemployment_Oct2012.csv',
    'US_Unemployment_Oct2012.csv',
     pd.read_csv)

unemployment =  dict(zip(unemployment['State'].tolist(), unemployment['Unemployment'].tolist()))
import json

In [ ]:
geo_json_data

In [ ]:
m = Map(center=(72,-45),
                zoom=2, max_zoom=5,
                basemap=basemaps.NASAGIBS.BlueMarble3413,
                crs=projections['EPSG:3413'])

In [ ]:
image = ImageOverlay(url=image_file, name="Speed", bounds=((72,-45), (73, -44)))
#m = Map(basemap=basemaps.NASAGIBS.BlueMarble3413, zoom=2, crs=its_live_epsg3413, center=(72,-45))
m.add_layer(image);
m

In [ ]:
raster_url = "/Users/andy/Google Drive/My Drive/data/ITS_LIVE/GRE_G0240_0000.nc"
ds = xr.open_dataset(raster_url, decode_times=False)

In [ ]:
wind = Velocity(
    data=ds,
    zonal_speed="vx",
    meridional_speed="vy",
    latitude_dimension="y",
    longitude_dimension="x",
    min_velocity=10,
    max_velocity=1500,
    display_options=display_options,
)
m.add(wind)

In [ ]:
def series_to_pathline_geopandas_dataframe(series: gp.GeoSeries, pathline: List) -> gp.GeoDataFrame:
    """
    Convert a list containing a pathline and a geopandas.GeoSeries to a geopandas.GeoDataFrame
    """

    k = series.name
    attributes = series.drop(columns="geometry").to_dict()
    del attributes["geometry"]

    points = pathline[0]
    v = pathline[1]
    vx = v[:, 0]
    vy = v[:, 1]
    speed = np.sqrt(vx**2 + vy**2)

    d = distances(points)
    pathline_data = {
        "vx": vx,
        "vy": vy,
        "v": speed,
        "pathline_id": k,
        "distance": d,
        "distance_from_origin": np.cumsum(d),
    }
    attributes.update(pathline_data)
    return pathline_to_geopandas_dataframe(points, attributes).reset_index(drop=True)


In [ ]:
from typing import List

In [ ]:
for k, df in starting_points.iterrows():
    print(df?)

In [ ]:
df.

In [ ]:
import plotly.express as px
df = px.data.gapminder().query("year == 2007")
fig = px.scatter_geo(df, locations="iso_alpha",
                     size="pop", # size of markers, "pop" is one of the columns of gapminder
                     )
fig.show()

In [ ]:
import plotly.express as px
df = px.data.gapminder().query("year == 2007")
fig = px.scatter_geo(df, locations="iso_alpha",
                     size="pop", # size of markers, "pop" is one of the columns of gapminder
                     )
fig.write_html('first_figure.html', auto_open=True)

In [ ]:
fig.show()

In [ ]:


from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

app = Dash(__name__)


app.layout = html.Div([
    html.H4('Interactive color selection with simple Dash example'),
    html.P("Select color:"),
    dcc.Dropdown(
        id="dropdown",
        options=['Gold', 'MediumTurquoise', 'LightGreen'],
        value='Gold',
        clearable=False,
    ),
    dcc.Graph(id="graph"),
])


@app.callback(
    Output("graph", "figure"), 
    Input("dropdown", "value"))
def display_color(color):
    fig = px.scatter_geo(pathlines_epsg4326_df,
                         lat=pathlines_epsg4326_df.geometry.y,
                         lon=pathlines_epsg4326_df.geometry.x,
                     )
    fig.update_geos(fitbounds="locations")
    return fig


app.run_server(debug=True)



In [ ]:
from dash_iconify import DashIconify
import dash_mantine_components as dmc

In [ ]:
px.scatter_geo?

In [ ]:
class LeafletMap(HasTraits):
    """A xarray.DataArray extension for interactive map plotting, based on ipyleaflet

    Parameters
    ----------
    ds : obj
        ``xarray.Dataset``

    Attributes
    ----------
    _ds : obj
        ``xarray.Dataset``
    _ds_selected : obj
        ``xarray.Dataset`` for selected variable
    _variable : str
        Selected variable
    map : obj
        ``ipyleaflet.Map``
    crs : str
        Coordinate Reference System of map
    left, top, right, bottom : float
        Map bounds in image coordinates
    sw : dict
        Location of lower-left corner in projected coordinates
    ne : dict
        Location of upper-right corner in projected coordinates
    bounds : tuple
        Location of map bounds in geographical coordinates
    image : obj
        ``ipyleaflet.ImageService`` layer for variable
    cmap : obj
        Matplotlib colormap object
    norm : obj
        Matplotlib normalization object
    opacity : float
        Transparency of image service layer
    colorbar : obj
        ``ipyleaflet.WidgetControl`` with Matplotlib colorbar
    popup : obj
        ``ipyleaflet.Popup`` with value at clicked location
    _data : float
        Variable value at clicked location
    _units : str
        Units of selected variable
    """

    bounds = Tuple(Tuple(Float(), Float()), Tuple(Float(), Float()))
    @observe('bounds')
    def boundary_change(self, change):
        """Update image on boundary change
        """
        # add image object to map
        if self._image is not None:
            # attempt to remove layer
            self.remove(self._image)
            # create new image service layer
            self._image = ipyleaflet.ImageService(
                name=self._variable,
                crs=self.crs,
                interactive=True,
                update_interval=100,
                endpoint='local')
        # add click handler for popups
        if self.enable_popups:
            self._image.on_click(self.handle_click)
        # set the image url
        self.set_image_url()
        self.add(self._image)

    def __init__(self, ds):
        # initialize map
        self.map = None
        self.crs = None
        self.left, self.top = (None, None)
        self.right, self.bottom = (None, None)
        self.sw = {}
        self.ne = {}
        # initialize dataset
        self._ds = ds
        self._ds_selected = None
        self._variable = None
        # initialize image and colorbars
        self._image = None
        self.cmap = None
        self.norm = None
        self.opacity = None
        self._colorbar = None
        # initialize attributes for popup
        self.enable_popups = False
        self._popup = None
        self._data = None
        self._units = None

    # add imagery data to leaflet map
    def plot(self, m, **kwargs):
        """Creates image plots on leaflet maps

        Parameters
        ----------
        m : obj
            leaflet map to add the layer
        variable : str, default 'delta_h'
            xarray variable to plot
        lag : int, default 0
            Time lag to plot if 3-dimensional
        cmap : str, default 'viridis'
            matplotlib colormap
        vmin : float or NoneType
            Minimum value for normalization
        vmax : float or NoneType
            Maximum value for normalization
        norm : obj or NoneType
            Matplotlib color normalization object
        opacity : float, default 1.0
            Opacity of image plot
        enable_popups : bool, default False
            Enable contextual popups
        colorbar : bool, decault True
            Show colorbar for rendered variable
        position : str, default 'topright'
            Position of colorbar on leaflet map
        """
        kwargs.setdefault('variable', 'delta_h')
        kwargs.setdefault('lag', 0)
        kwargs.setdefault('cmap', 'viridis')
        kwargs.setdefault('vmin', None)
        kwargs.setdefault('vmax', None)
        kwargs.setdefault('norm', None)
        kwargs.setdefault('opacity', 1.0)
        kwargs.setdefault('enable_popups', False)
        kwargs.setdefault('colorbar', True)
        kwargs.setdefault('position', 'topright')
        # set map and map coordinate reference system
        self.map = m
        crs = m.crs['name']
        self.crs = projections[crs]
        (self.left, self.top), (self.right, self.bottom) = self.map.pixel_bounds
        # enable contextual popups
        self.enable_popups = bool(kwargs['enable_popups'])
        # reduce to variable and lag
        self._variable = copy.copy(kwargs['variable'])
        self.lag = int(kwargs['lag'])
        if (self._ds[self._variable].ndim == 3) and ('time' in self._ds[self._variable].dims):
            self._ds_selected = self._ds[self._variable].sel(time=self._ds.time[self.lag])
        elif (self._ds[self._variable].ndim == 3) and ('band' in self._ds[self._variable].dims):
            self._ds_selected = self._ds[self._variable].sel(band=1)
        else:
            self._ds_selected = self._ds[self._variable]
        # get the normalization bounds
        self.get_norm_bounds(**kwargs)
        # create matplotlib normalization
        if kwargs['norm'] is None:
            self.norm = colors.Normalize(vmin=self.vmin, vmax=self.vmax, clip=True)
        else:
            self.norm = copy.copy(kwargs['norm'])
        # get colormap
        self.cmap = copy.copy(cm.get_cmap(kwargs['cmap']))
        # get opacity
        self.opacity = float(kwargs['opacity'])
        # wait for changes
        asyncio.ensure_future(self.async_wait_for_bounds())
        self._image = ipyleaflet.ImageService(
            name=self._variable,
            crs=self.crs,
            interactive=True,
            update_interval=100,
            endpoint='local')
        # add click handler for popups
        if self.enable_popups:
            self._image.on_click(self.handle_click)
        # set the image url
        self.set_image_url()
        # add image object to map
        self.add(self._image)
        # add colorbar
        self.colorbar = kwargs['colorbar']
        self.colorbar_position = kwargs['position']
        if self.colorbar:
            self.add_colorbar(
                label=self._variable,
                cmap=self.cmap,
                opacity=self.opacity,
                norm=self.norm,
                position=self.colorbar_position
            )

    def wait_for_change(self, widget, value):
        future = asyncio.Future()
        def get_value(change):
            future.set_result(change.new)
            widget.unobserve(get_value, value)
        widget.observe(get_value, value)
        return future

    async def async_wait_for_bounds(self):
        if len(self.map.bounds) == 0:
            await self.wait_for_change(self.map, 'bounds')
        # check that bounds are close
        while True:
            self.get_bounds()
            await self.wait_for_change(self.map, 'bounds')
            if np.isclose(self.bounds, self.map.bounds).all():
                break
        # will update map

    def add(self, obj):
        """wrapper function for adding layers and controls to leaflet maps
        """
        try:
            self.map.add(obj)
        except ipyleaflet.LayerException as exc:
            logging.info(f"{obj} already on map")
            pass
        except ipyleaflet.ControlException as exc:
            logging.info(f"{obj} already on map")
            pass

    def remove(self, obj):
        """wrapper function for removing layers and controls to leaflet maps
        """
        try:
            self.map.remove(obj)
        except ipyleaflet.LayerException as exc:
            logging.info(f"{obj} already removed from map")
            pass
        except ipyleaflet.ControlException as exc:
            logging.info(f"{obj} already removed from map")
            pass

    @property
    def z(self):
        """get the map zoom level
        """
        return int(self.map.zoom)

    @property
    def resolution(self):
        """get the map resolution for a given zoom level
        """
        return self.map.crs['resolutions'][self.z]

    def reset(self):
        """remove features from leaflet map
        """
        for layer in self.map.layers:
            if (layer._model_name == 'LeafletImageServiceModel') and \
                (layer.endpoint == 'local'):
                self.remove(layer)
            elif (layer._model_name == 'LeafletPopupModel'):
                self.remove(layer)
        for control in self.map.controls:
            if (control._model_name == 'LeafletWidgetControlModel') and \
                (control.widget._model_name == 'ImageModel'):
                self.remove(control)
        # reset layers and controls
        self._image = None
        self._popup = None
        self._colorbar = None

    # get map bounding box in projected coordinates
    def get_bbox(self):
        """get the bounding box of the leaflet map in projected coordinates
        """
        # get SW and NE corners in map coordinates
        (self.left, self.top), (self.right, self.bottom) = self.map.pixel_bounds
        self.sw = dict(x=(self.map.crs['origin'][0] + self.left*self.resolution),
            y=(self.map.crs['origin'][1] - self.bottom*self.resolution))
        self.ne = dict(x=(self.map.crs['origin'][0] + self.right*self.resolution),
            y=(self.map.crs['origin'][1] - self.top*self.resolution))
        return self

    # get map bounds in geographic coordinates
    def get_bounds(self):
        """get the bounds of the leaflet map in geographical coordinates
        """
        self.get_bbox()
        lon, lat = rasterio.warp.transform(
            self.crs['name'], 'EPSG:4326',
            [self.sw['x'], self.ne['x']],
            [self.sw['y'], self.ne['y']])
        # calculate bounds in latitude/longitude
        north = np.max(lat)
        east = np.max(lon)
        south = np.min(lat)
        west = np.min(lon)
        # update bounds
        self.bounds = ((south, west), (north, east))

    def get_crs(self):
        """Attempt to get the coordinate reference system of the dataset
        """
        # get coordinate reference system from grid mapping
        try:
            grid_mapping = self._ds[self._variable].attrs['grid_mapping']
            ds_crs = self._ds[grid_mapping].attrs['crs_wkt']
        except Exception as exc:
            pass
        else:
            self._ds.rio.set_crs(ds_crs)
            return
        # get coordinate reference system from crs attribute
        try:
            ds_crs = self._ds.rio.crs.to_wkt()
        except Exception as exc:
            pass
        else:
            self._ds.rio.set_crs(ds_crs)
            return
        # raise exception
        raise Exception('Unknown coordinate reference system')
    
    def get_norm_bounds(self, **kwargs):
        """
        Get the colorbar normalization bounds

        Parameters
        ----------
        vmin : float or NoneType
            Minimum value for normalization
        vmax : float or NoneType
            Maximum value for normalization
        """
        # set default keyword arguments
        kwargs.setdefault('vmin', None)
        kwargs.setdefault('vmax', None)
        # set colorbar limits to 2-98 percentile
        # if not using a defined plot range
        clim = self._ds_selected.chunk(dict(y=-1,x=-1)).quantile((0.02, 0.98)).values
        # set minimum for normalization
        fmin = np.finfo(np.float64).min
        if (kwargs['vmin'] is None) or np.isclose(kwargs['vmin'], fmin):
            self.vmin = clim[0]
            self._dynamic = True
        else:
            self.vmin = np.copy(kwargs['vmin'])
            self._dynamic = False
        # set maximum for normalization
        fmax = np.finfo(np.float64).max
        if (kwargs['vmax'] is None) or np.isclose(kwargs['vmax'], fmax):
            self.vmax = clim[-1]
            self._dynamic = True
        else:
            self.vmax = np.copy(kwargs['vmax'])
            self._dynamic = False

    def validate_norm(self):
        """
        Validate the colorbar normalization bounds
        """
        fmin = np.finfo(np.float64).min
        fmax = np.finfo(np.float64).max
        if np.isclose(self.vmin, fmin):
            self.vmin = -5
            self._dynamic = False
        if np.isclose(self.vmax, fmax):
            self.vmax = 5
            self._dynamic = False

    def clip_image(self, ds):
        """clip or warp xarray image to bounds of leaflet map
        """
        self.get_bbox()
        # attempt to get the coordinate reference system of the dataset
        self.get_crs()
        # convert map bounds to coordinate reference system of image
        minx, miny, maxx, maxy = rasterio.warp.transform_bounds(
            self.crs['name'], self._ds.rio.crs,
            self.sw['x'], self.sw['y'],
            self.ne['x'], self.ne['y'])
        # extent of the leaflet map
        self.extent = np.array([minx, maxx, miny, maxy])
        # compare data resolution and leaflet map resolution
        resolution = np.abs(ds.x[1] - ds.x[0]).values
        if (resolution > self.resolution):
            # pad input image to map bounds
            padded = ds.rio.pad_box(minx=minx, maxx=maxx, miny=miny, maxy=maxy)
            # get affine transform of padded image
            pad_transform = padded.rio.transform()
            north = int((maxy - pad_transform.f)//pad_transform.e)
            east = int((maxx - pad_transform.c)//pad_transform.a) + 1
            south = int((miny - pad_transform.f)//pad_transform.e) + 1
            west = int((minx - pad_transform.c)//pad_transform.a)
            # clip image to map bounds
            return padded.isel(x=slice(west, east), y=slice(north, south))
        else:
            # warp image to map bounds and resolution
            # input and output affine transformations
            src_transform = ds.rio.transform()
            dst_transform = rasterio.transform.from_origin(minx, maxy,
                self.resolution, self.resolution)
            # allocate for output warped image
            dst_width = int((maxx - minx)//self.resolution)
            dst_height = int((maxy - miny)//self.resolution)
            dst_data = np.zeros((dst_height, dst_width), dtype=ds.dtype.type)
            # warp image to output resolution
            rasterio.warp.reproject(source=ds.values, destination=dst_data,
                src_transform=src_transform,
                src_crs=self._ds.rio.crs,
                src_nodata=np.nan,
                dst_transform=dst_transform,
                dst_crs=self.crs['name'],
                dst_resolution=(self.resolution, self.resolution))
            # calculate centered coordinates
            transform = dst_transform * dst_transform.translation(0.5, 0.5)
            x_coords, _ = transform * (np.arange(dst_width), np.zeros(dst_width))
            _, y_coords = transform * (np.zeros(dst_height), np.arange(dst_height))
            # return DataAarray with warped image
            return xr.DataArray(
                name=ds.name,
                data=dst_data,
                coords=dict(y=y_coords, x=x_coords),
                dims=copy.deepcopy(ds.dims),
                attrs=copy.deepcopy(ds.attrs),
            )

    def get_image_url(self):
        """create the image url for the imageservice
        """
        fig, ax = plt.subplots(1, figsize=(15, 8))
        fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
        visible = self.clip_image(self._ds_selected)
        visible.plot.imshow(ax=ax,
            norm=self.norm,
            interpolation="nearest",
            cmap=self.cmap,
            alpha=self.opacity,
            add_colorbar=False,
            add_labels=False
        )
        # set image extent
        ax.set_xlim(self.extent[0], self.extent[1])
        ax.set_ylim(self.extent[2], self.extent[3])
        ax.axis("tight")
        ax.axis("off")
        # save as in-memory png
        png = io.BytesIO()
        plt.savefig(png, format='png', transparent=True)
        plt.close()
        png.seek(0)
        # encode to base64 and get url
        data = base64.b64encode(png.read()).decode("ascii")
        self.url = "data:image/png;base64," + data
        return self

    def set_image_url(self, *args, **kwargs):
        """set the url for the imageservice
        """
        self.get_bounds()
        self.get_image_url()
        self._image.url = self.url

    def redraw(self, *args, **kwargs):
        """
        Redraw the image on the map
        """
        # try to update the selected dataset
        try:
            self.get_image_url()
        except Exception as exc:
            pass
        else:
            # update image url
            self._image.url = self.url
            # force redrawing of map by removing and adding layer
            self.remove(self._image)
            self.add(self._image)

    def redraw_colorbar(self, *args, **kwargs):
        """
        Redraw the colorbar on the map
        """
        try:
            if self.colorbar:
                self.add_colorbar(
                    label=self._variable,
                    cmap=self.cmap,
                    opacity=self.opacity,
                    norm=self.norm,
                    position=self.colorbar_position
                )
        except Exception as exc:
            pass

    # observe changes in widget parameters
    def set_observables(self, widget, **kwargs):
        """observe changes in widget parameters
        """
        # set default keyword arguments
        # to map widget changes to functions
        kwargs.setdefault('variable', [self.set_variable])
        kwargs.setdefault('timelag', [self.set_lag])
        kwargs.setdefault('range', [self.set_norm])
        kwargs.setdefault('dynamic', [self.set_dynamic])
        kwargs.setdefault('cmap', [self.set_colormap])
        kwargs.setdefault('reverse', [self.set_colormap])
        # connect each widget with a set function
        for key, val in kwargs.items():
            # try to retrieve the functional
            try:
                observable = getattr(widget, key)
            except AttributeError as exc:
                continue
            # assert that observable is an ipywidgets object
            assert isinstance(observable, ipywidgets.widgets.widget.Widget)
            assert hasattr(observable, 'observe')
            # for each functional to map
            for i, functional in enumerate(val):
                # try to connect the widget to the functional
                try:
                    observable.observe(functional)
                except (AttributeError, NameError, ValueError) as exc:
                    pass

    def set_variable(self, sender):
        """update the dataframe variable for a new selected variable
        """
        # only update variable if a new final
        if isinstance(sender['new'], str):
            self._variable = sender['new']
        else:
            return
        # reduce to variable and lag
        if (self._ds[self._variable].ndim == 3) and ('time' in self._ds[self._variable].dims):
            self._ds_selected = self._ds[self._variable].sel(time=self._ds.time[self.lag])
        elif (self._ds[self._variable].ndim == 3) and ('band' in self._ds[self._variable].dims):
            self._ds_selected = self._ds[self._variable].sel(band=1)
        else:
            self._ds_selected = self._ds[self._variable]
        # check if dynamic normalization is enabled
        if self._dynamic:
            self.get_norm_bounds()
            self.norm.vmin = self.vmin
            self.norm.vmax = self.vmax
        # try to redraw the selected dataset
        self.redraw()
        self.redraw_colorbar()

    def set_lag(self, sender):
        """update the time lag for the selected variable
        """
        # only update lag if a new final
        if isinstance(sender['new'], int):
            self.lag = sender['new'] - 1
        else:
            return
        # try to update the selected dataset
        self._ds_selected = self._ds[self._variable].sel(time=self._ds.time[self.lag])
        # check if dynamic normalization is enabled
        if self._dynamic:
            self.get_norm_bounds()
            self.norm.vmin = self.vmin
            self.norm.vmax = self.vmax
        # try to redraw the selected dataset
        self.redraw()
        if self._dynamic:
            self.redraw_colorbar()

    def set_dynamic(self, sender):
        """set dynamic normalization for the selected variable
        """
        # only update dynamic norm if a new final
        if isinstance(sender['new'], bool) and sender['new']:
            self.get_norm_bounds()
            self._dynamic = True
        elif isinstance(sender['new'], bool):
            self.vmin, self.vmax = (-5, 5)
            self._dynamic = False
        else:
            return
        # set the normalization bounds
        self.validate_norm()
        self.norm.vmin = self.vmin
        self.norm.vmax = self.vmax
        # try to redraw the selected dataset
        self.redraw()
        self.redraw_colorbar()

    def set_norm(self, sender):
        """update the normalization for the selected variable
        """
        # only update norm if a new final
        if isinstance(sender['new'], (tuple, list)):
            self.vmin, self.vmax = sender['new']
        else:
            return
        # set the normalization bounds
        self.validate_norm()
        self.norm.vmin = self.vmin
        self.norm.vmax = self.vmax
        # try to redraw the selected dataset
        self.redraw()
        self.redraw_colorbar()

    def set_colormap(self, sender):
        """update the colormap for the selected variable
        """
        # only update colormap if a new final
        if isinstance(sender['new'], str):
            cmap_name = self.cmap.name
            cmap_reverse_flag = '_r' if cmap_name.endswith('_r') else ''
            self.cmap = cm.get_cmap(sender['new'] + cmap_reverse_flag)
        elif isinstance(sender['new'], bool):
            cmap_name = self.cmap.name.strip('_r')
            cmap_reverse_flag = '_r' if sender['new'] else ''
            self.cmap = cm.get_cmap(cmap_name + cmap_reverse_flag)
        else:
            return
        # try to redraw the selected dataset
        self.redraw()
        self.redraw_colorbar()

    # functional calls for click events
    def handle_click(self, **kwargs):
        """callback for handling mouse clicks
        """
        lat, lon = kwargs.get('coordinates')
        # remove any prior instances of popup
        if self._popup is not None:
            self.remove(self._popup)
        # attempt to get the coordinate reference system of the dataset
        try:
            grid_mapping = self._ds[self._variable].attrs['grid_mapping']
            crs = self._ds[grid_mapping].attrs['crs_wkt']
        except Exception as exc:
            crs = self._ds.rio.crs.to_wkt()
        else:
            self._ds.rio.set_crs(crs)
        # get the clicked point in dataset coordinate reference system
        x, y = rasterio.warp.transform('EPSG:4326', crs, [lon], [lat])
        # find nearest point in dataset
        self._data = self._ds_selected.sel(x=x, y=y, method='nearest').values[0]
        self._units = self._ds[self._variable].attrs['units']
        # only create popup if valid
        if np.isnan(self._data):
            return
        # create contextual popup
        child = ipywidgets.HTML()
        child.value = '{0:0.1f} {1}'.format(np.squeeze(self._data), self._units)
        self._popup = ipyleaflet.Popup(location=(lat, lon),
            child=child, name='popup')
        self.add(self._popup)

    # add colorbar widget to leaflet map
    def add_colorbar(self, **kwargs):
        """Creates colorbars on leaflet maps

        Parameters
        ----------
        cmap : str, matplotlib colormap
        norm : obj, matplotlib color normalization object
        opacity : float, opacity of colormap
        orientation : str, orientation of colorbar
        label : str, label for colorbar
        position : str, position of colorbar on leaflet map
        width : float, width of colorbar
        height : float, height of colorbar
        """
        kwargs.setdefault('cmap', 'viridis')
        kwargs.setdefault('norm', None)
        kwargs.setdefault('opacity', 1.0)
        kwargs.setdefault('orientation', 'vertical')
        kwargs.setdefault('label', 'delta_h')
        kwargs.setdefault('position', 'topright')
        kwargs.setdefault('width', 0.2)
        kwargs.setdefault('height', 3.0)
        # remove any prior instances of a colorbar
        if self._colorbar is not None:
            self.remove(self._colorbar)
        # create matplotlib colorbar
        _, ax = plt.subplots(figsize=(kwargs['width'], kwargs['height']))
        cbar = matplotlib.colorbar.ColorbarBase(ax,
            cmap=kwargs['cmap'],
            norm=kwargs['norm'],
            alpha=kwargs['opacity'],
            orientation=kwargs['orientation'],
            label=kwargs['label'])
        cbar.solids.set_rasterized(True)
        cbar.ax.tick_params(which='both', width=1, direction='in')
        # save colorbar to in-memory png object
        png = io.BytesIO()
        plt.savefig(png, bbox_inches='tight', pad_inches=0.075,
            format='png', transparent=True)
        png.seek(0)
        # create output widget
        output = ipywidgets.Image(value=png.getvalue(), format='png')
        self._colorbar = ipyleaflet.WidgetControl(widget=output,
            transparent_bg=False, position=kwargs['position'])
        # add colorbar
        self.add(self._colorbar)
        plt.close()

    # save the current map as an image
    def imshow(self, ax=None, **kwargs):
        """Save the current map as a static image

        Parameters
        ----------
        ax: obj, default None
            Figure axis
        kwargs: dict, default {}
            Additional keyword arguments for ``imshow``
        """
        # create figure axis if non-existent
        if (ax is None):
            _, ax = plt.subplots()
        # extract units
        longname = self._ds[self._variable].attrs['long_name'].replace('  ', ' ')
        units = self._ds[self._variable].attrs['units'][0]
        # clip image to map bounds
        visible = self.clip_image(self._ds_selected)
        # color bar keywords
        cbar_kwargs = dict(label=f'{longname} [{units}]', orientation='horizontal')
        visible.plot.imshow(ax=ax,
            norm=self.norm,
            interpolation="nearest",
            cmap=self.cmap,
            alpha=self.opacity,
            add_colorbar=True,
            add_labels=True,
            cbar_kwargs=cbar_kwargs,
            **kwargs
        )
        # set image extent
        ax.set_xlim(self.extent[0], self.extent[1])
        ax.set_ylim(self.extent[2], self.extent[3])
        ax.set_aspect('equal', adjustable='box')


In [ ]:
class FileTree:

    def __init__(self, filepath: os.PathLike):
        """
        Usage: component = FileTree('Path/to/my/File').render()
        """
        self.filepath = filepath

    def render(self) -> dmc.Accordion:
        return dmc.Accordion(
            self.build_tree(self.filepath, isRoot=True), multiple=True)

    def flatten(self, l):
        return [item for sublist in l for item in sublist]

    def make_file(self, file_name):
        return dmc.Text(
            [DashIconify(icon="akar-icons:file"), " ", file_name], style={"paddingTop": '5px'}
        )

    def make_folder(self, folder_name):
        return [DashIconify(icon="akar-icons:folder"), " ", folder_name]

    def build_tree(self, path, isRoot=False):
        d = []
        if os.path.isdir(path):
            children = self.flatten([self.build_tree(os.path.join(path, x))
                                    for x in os.listdir(path)])

In [ ]:
px.scatter?

In [ ]:
component = FileTree('Path/to/my/Directory').render()

In [ ]:
app.run_server(component)

In [ ]:
df = px.data.gapminder().query("year == 2007")
fig = px.scatter_geo(df, locations="iso_alpha",
                     size="pop", # size of markers, "pop" is one of the columns of gapminder
                     )

In [ ]:
    fig = px.scatter(df, x="iso_alpha", y="gdpPercap",
                     )
    fig.show()

In [ ]:
import plotly.express as px
#px.set_mapbox_access_token(open(".mapbox_token").read())
df = px.data.carshare()
fig = px.scatter_mapbox(df, lat="centroid_lat", lon="centroid_lon",     color="peak_hour", size="car_hours",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10, mapbox_style="stamen-terrain")
fig.show()

In [ ]:
px.scatter_mapbox?

In [ ]:
import numpy as np
np.seterr(all="ignore")

nx = 201
ny = 401
x = np.linspace(-100e3, 100e3, nx)
y = np.linspace(-100e3, 100e3, ny)
X, Y = np.meshgrid(x, y)

vx = -Y / np.sqrt(X**2 + Y**2) * 250
vy = X / np.sqrt(X**2 + Y**2) * 250
p = [0, -50000]


In [ ]:
np.seterr(divide="ignore")

In [ ]:
from glacier_flow_tools.pathlines import compute_pathline, pathline_to_geopandas_dataframe
from glacier_flow_tools.geom import distances

In [ ]:
pathline = compute_pathline(p, vx, vx, x, y, dt=1, total_time=10)

In [ ]:
pathline_to_geopandas_dataframe(pts).to_dict()

In [ ]:
    vx = v[:, 0]
    vy = v[:, 1]
    speed = np.sqrt(vx**2 + vy**2)

    d = distances(pts)
    pathline_data = {
        "vx": vx,
        "vy": vy,
        "v": speed,
        "distance": d,
        "distance_from_origin": np.cumsum(d),
    }
    attributes.update(pathline_data)


In [ ]:
pl = pathline_to_geopandas_dataframe(pts, attributes)

In [ ]:
attributes = {"pathline_id": 0}

In [ ]:
pl.to_dict()

In [ ]:
compute_pathline(p, vx, vx, x, y, dt=1, total_time=10)

In [ ]:
x

In [ ]:
y

In [ ]:
pathline

In [ ]:
pathline = (np.array([[   249.99370971, -49750.00629029],
        [   499.97467017, -49500.02532983],
        [   749.94262324, -49250.05737676],
        [   999.89730564, -49000.10269436],
        [  1249.82538394, -48750.17461606],
        [  1499.71314778, -48500.28685222],
        [  1749.55977134, -48250.44022866],
        [  1999.36441369, -48000.63558631],
        [  2249.11273931, -47750.88726069],
        [  2498.78991573, -47501.21008427],
        [  2748.39446997, -47251.60553003]]),
 np.array([[249.98737725, 249.98737725],
        [249.97450152, 249.97450152],
        [249.96136118, 249.96136118],
        [249.94796018, 249.94796018],
        [249.90805497, 249.90805497],
        [249.8673395 , 249.8673395 ],
        [249.82577025, 249.82577025],
        [249.78337719, 249.78337719],
        [249.71298834, 249.71298834],
        [249.64113141, 249.64113141],
        [249.56773625, 249.56773625]]),
 np.array([[2.84217094e-14],
        [0.00000000e+00],
        [7.58982048e-12],
        [0.00000000e+00],
        [4.19384810e-08],
        [2.27373675e-13],
        [1.87642803e-10],
        [2.27373675e-13],
        [2.69376538e-07],
        [4.54747351e-13],
        [2.25377857e-09]]))

In [ ]:
p

In [ ]:
starting_points.loc[0].to_dict()

In [ ]:
series = gp.GeoSeries([Point([0, -5000])])

In [ ]:
series_to_pathline_geopandas_dataframe(series, pathline)

In [ ]:
series

In [ ]:
series.to_dict()

In [ ]:
def series_to_pathline_geopandas_dataframe(series: gp.GeoSeries, pathline: Tuple) -> gp.GeoDataFrame:
    """
    Convert a  pathline tuple and a geopandas.GeoSeries to a geopandas.GeoDataFrame

    Convert a geopandas.GeoSeries single point and a pathline to a geopandas.GeoDataFrame
    
    Parameters
    ----------
    series : geopandas.GeoSeries
        A series with attributes
    pathline : Tuple
        Tuple of (List[Points], List[vx, vy], List[error])

    Returns
    ----------
    df: gp.GeoDataFrame
        Geopandas dataframe of pathline

    Examples
    ----------

    Create data:

    >>>    import numpy as np
    >>>    import geopandas as gp
    >>>    from glacier_flow_tools.pathlines import series_to_pathline_geopandas_dataframe

    >>>    series = gp.GeoSeries([Point([0, -5000])])
    >>>    pathline = (np.array([[   249.99370971, -49750.00629029],
    >>>            [   499.97467017, -49500.02532983],
    >>>            [   749.94262324, -49250.05737676],
    >>>            [   999.89730564, -49000.10269436],
    >>>            [  1249.82538394, -48750.17461606],
    >>>            [  1499.71314778, -48500.28685222],
    >>>            [  1749.55977134, -48250.44022866],
    >>>            [  1999.36441369, -48000.63558631],
    >>>            [  2249.11273931, -47750.88726069],
    >>>            [  2498.78991573, -47501.21008427],
    >>>            [  2748.39446997, -47251.60553003]]),
    >>>     np.array([[249.98737725, 249.98737725],
    >>>            [249.97450152, 249.97450152],
    >>>            [249.96136118, 249.96136118],
    >>>            [249.94796018, 249.94796018],
    >>>            [249.90805497, 249.90805497],
    >>>            [249.8673395 , 249.8673395 ],
    >>>            [249.82577025, 249.82577025],
    >>>            [249.78337719, 249.78337719],
    >>>            [249.71298834, 249.71298834],
    >>>            [249.64113141, 249.64113141],
    >>>            [249.56773625, 249.56773625]]),
    >>>     np.array([[2.84217094e-14],
    >>>            [0.00000000e+00],
    >>>            [7.58982048e-12],
    >>>            [0.00000000e+00],
    >>>            [4.19384810e-08],
    >>>            [2.27373675e-13],
    >>>            [1.87642803e-10],
    >>>            [2.27373675e-13],
    >>>            [2.69376538e-07],
    >>>            [4.54747351e-13],
    >>>            [2.25377857e-09]]))    
    """

    k = series.name
    attributes = series.drop(columns="geometry").to_dict()
    if hasattr(attributes, "geometry"):
        del attributes["geometry"]

    points = pathline[0]
    v = pathline[1]
    vx = v[:, 0]
    vy = v[:, 1]
    speed = np.sqrt(vx**2 + vy**2)

    d = distances(points)
    pathline_data = {
        "vx": vx,
        "vy": vy,
        "v": speed,
        "pathline_id": k,
        "distance": d,
        "distance_from_origin": np.cumsum(d),
    }
    attributes.update(pathline_data)
    return pathline_to_geopandas_dataframe(points, attributes).reset_index(drop=True)


In [ ]:
from typing import Tuple

In [ ]:
from glacier_flow_tools.gaussian_random_fields import distrib_normal, power_spectrum

In [ ]:
da = np.array([[0., 1.],[1.0, 0.]])
distrib_normal(da)

In [ ]:
power_spectrum(da, 1)

In [ ]:
    def exact_solution(x0: np.ndarray, t: float) -> np.ndarray:
        """
        Exact solution for linear flow.

        Parameters
        ----------
        x0 : np.ndarray
            Initial position.
        t : np.ndarray
            Time.

        Returns
        -------

        np.ndarray
          Exact position after time t.

        Examples
        --------
        FIXME: Add docs.
        """
        return x0 * np.exp([t, -t])


In [ ]:
    total_time = 1.0
    starting_point = [0.05, 0.95]

    r_exact = exact_solution(starting_point, total_time)

In [ ]:
r_exact

In [ ]:
ds.drop_vars?